## Rationale
We want to build a small system to handle different tasks asynchronously.

There will be a main `loop` keep checking a `queue` for `task`.
It will take the first `task` on the `queue` and run it.
The `task` that runs should do part of its job, suspends, then return to the `main loop`.
Because we will need to resume to that `suspended task`, we need to put the `task` into the `queue` again.
The `main loop` then move to the next `task`.
We need to handle when the `task` really comes to the end.

In [1]:
import asyncio
from queue import Queue

In [2]:
class Task:
    task_id = 1
    def __init__(self, coro):
        print('Initiating coro:', coro)
        self._coro = coro
        self.id = Task.task_id
        Task.task_id += 1
        
    def run(self, value):
        return self._coro.send(value)
        
def simple_task(msg):
    print('simple_task {} started'.format(msg))
    print("I'm only do simple", msg)
    yield from wait_for(random.randrange(1, 5))
    print('Closing simple task')

def wait_for(time):
    for i in range(time):
        print('zzzzzzzzzzzzzzzzzzzzzsleepingzzzzzzzzzzzzzzzzzzzzzzzz')
        yield
        
def io_operation():
    for i in range(10):
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
        print("I'm blocking")
        print("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx")
        yield
        
    return 42

def task_with_io():
    print("About to do some io stuff")
    result = yield from io_operation()
    print('Result:', result)
    
def parent_task(msg):
    print("I'm a parent task with msg:", msg)
    yield NewTask(simple_task("nested task with msg" + msg))
    print("I gave birth to my child task, my work here is done")

In [3]:
class SystemCall:
    pass

class NewTask(SystemCall):
    
    def __init__(self, target):
        self.target = target
        
    def handle(self, scheduler, calling_task):
        task_id = scheduler.new(self.target)
        scheduler.schedule(calling_task)
        
        return task_id
    
class ReadWait(SystemCall):
    
    def __init__(self, target):
        self.target = target
        
    def handle(self, scheduler, calling_task):
        scheduler.waitforread(calling_task, self.target)

In [4]:
import asyncio
import random
from time import sleep
from queue import Queue


class Scheduler:
    
    def __init__(self):
        self._queue = Queue()  # This is our tasks queue
        self.read = {}
        self.write = {}
        
    def new(self, target):
        new_task = Task(target)
        self.schedule(new_task)
        
        return new_task.id
        
    def schedule(self, task):
        self._queue.put(task)  # append the tasks to queue
        
    def waitforread(self, task, fd):
        self.read[fd] = task
        
    def waitforwrite(self, task, fd):
        self.write[fd] = task
        
    def run(self):
        # The main body of the scheduler that will run task
        for i in range(20):
            print('---------------------------------Running cycle-------------------------------', i)
            if not self._queue.empty():
                current_task = self._queue.get()
            else:
                print('The queue is exhausted')
                continue

            try:
                result = current_task.run(None)
                print(result)
                if isinstance(result, SystemCall):
                    print('Handling systemcall')
                    result.handle(self, current_task)
                    continue
            except StopIteration:
                print('Task ended')
            else:
                self.schedule(current_task)
            


    
scheduler = Scheduler()

task3 = scheduler.new(task_with_io())
task4 = scheduler.new(parent_task('From outter'))

scheduler.run()

Initiating coro: <generator object task_with_io at 0x0000026536E02350>
Initiating coro: <generator object parent_task at 0x0000026536E02660>
---------------------------------Running cycle------------------------------- 0
About to do some io stuff
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
I'm blocking
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
None
---------------------------------Running cycle------------------------------- 1
I'm a parent task with msg: From outter
Handling systemcall
Initiating coro: <generator object simple_task at 0x0000026536E024A0>
---------------------------------Running cycle------------------------------- 2
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
I'm blocking
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
None
---------------------------------Running cycle------------------------------- 3
simple_task nested task with msgFrom outter started
I'm only do simple nested task with msgFrom outter
zzzzzzzzzzzzzzzzzzzzzsleepingzzzzzzzzzzzzzzzzzzzzzzzz
None
-------------------------

In [5]:
from socket import socket, AF_INET, SOCK_STREAM

def handle_client(client, addr):
    print('Connection from', addr)
    while True:
        print('Waiting for client input')
        data = client.recv(65536)
        print(data)
        if data == b'\x03':
            break
        client.send(data)
    client.close()
    print('Client closed')
    yield

def server(port=2323):
    print('Server starting')
    sock = socket(AF_INET, SOCK_STREAM)
    sock.bind(("", port))
    sock.listen(5)
    while True:
        try:
            yield ReadWait(sock)
            client, addr = sock.accept()
            yield NewTask(handle_client(client, addr))
        except KeyboardInterrupt:
            break

In [6]:
scheduler = Scheduler()

task3 = scheduler.new(server())


scheduler.run()

Initiating coro: <generator object server at 0x0000026536E02580>
---------------------------------Running cycle------------------------------- 0
Server starting
Handling systemcall
---------------------------------Running cycle------------------------------- 1
The queue is exhausted
---------------------------------Running cycle------------------------------- 2
The queue is exhausted
---------------------------------Running cycle------------------------------- 3
The queue is exhausted
---------------------------------Running cycle------------------------------- 4
The queue is exhausted
---------------------------------Running cycle------------------------------- 5
The queue is exhausted
---------------------------------Running cycle------------------------------- 6
The queue is exhausted
---------------------------------Running cycle------------------------------- 7
The queue is exhausted
---------------------------------Running cycle------------------------------- 8
The queue is exhau